In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing,tree


In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
train_df[train_df["target"] == 0]["text"].values[3]

'My car is so fast'

In [6]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_df["text"])
Y = vectorizer.transform(test_df["text"])

In [7]:
vectorizer.get_feature_names_out()


array(['00', '000', '0000', ..., 'ûónegligence', 'ûótech', 'ûówe'],
      dtype=object)

In [8]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=4,         
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(train_df)

In [9]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, X, train_df["target"], cv=3, scoring="f1")
scores

array([0.63366337, 0.6122449 , 0.68442211])

In [10]:
clf.fit(X, train_df["target"])

RidgeClassifier()

In [11]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = clf.predict(Y)

In [12]:
sample_submission["target"] = clf.predict(Y)

In [13]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
